In [2]:
import pandas as pd
import numpy as np

In [8]:
# load full_info_dataframe.csv
data_full = pd.read_csv('../merged_data/full_info_dataframe.csv', sep=",")

/var/folders/7_/wb06s1193t5fbc2j9sggjlj80000gn/T/ipykernel_46762/59176.py:2: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  data_full = pd.read_csv('../merged_data/full_info_dataframe.csv', sep=",")


In [12]:
data_full.head()

,client_id,visitor_id,visit_id,process_step,date_time,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,Variation
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51,5.0,64.0,79.0,NaN,2.0,189023.86,1.0,4.0,Test
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22,5.0,64.0,79.0,NaN,2.0,189023.86,1.0,4.0,Test
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13,5.0,64.0,79.0,NaN,2.0,189023.86,1.0,4.0,Test
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04,5.0,64.0,79.0,NaN,2.0,189023.86,1.0,4.0,Test
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:17:15,5.0,64.0,79.0,NaN,2.0,189023.86,1.0,4.0,Test


In [11]:
data_full['Variation'].unique()

array(['Test', 'Control', nan], dtype=object)

In [14]:
data_full['process_step'].unique()

array(['step_2', 'step_3', 'step_1', 'start', 'confirm'], dtype=object)

In [15]:
# Remove rows where 'Variation' is NaN (we only want 'Test' and 'Control' users)
data_full = data_full.dropna(subset=['Variation'])

In [17]:
# Convert the 'date_time' column to datetime format for time calculations
data_full['date_time'] = pd.to_datetime(data_full['date_time'])

/var/folders/7_/wb06s1193t5fbc2j9sggjlj80000gn/T/ipykernel_46762/1595391514.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_full['date_time'] = pd.to_datetime(data_full['date_time'])


In [18]:
# Sort the data by user and timestamp to ensure correct time difference calculations
data_sorted = data_full.sort_values(by=['visitor_id', 'visit_id', 'date_time'])

In [19]:
# Calculate the time difference between each step and the next step within each visit
# This gives the time spent on the current step before moving to the next one
data_sorted['time_diff'] = data_sorted.groupby(['visitor_id', 'visit_id'])['date_time'].diff().shift(-1)

In [21]:
# Separate the data into Test and Control groups
df_test = data_sorted[data_sorted['Variation'] == 'Test']
df_control = data_sorted[data_sorted['Variation'] == 'Control']

In [22]:
# Group by process_step and calculate the average time spent on each step (for Test group)
avg_time_test = df_test.groupby('process_step')['time_diff'].mean()

In [23]:
# Group by process_step and calculate the average time spent on each step (for Control group)
avg_time_control = df_control.groupby('process_step')['time_diff'].mean()

In [33]:
# Function to format timedelta into MM:SS
def format_timedelta(td):
    if pd.isnull(td):
        return None
    total_seconds = int(td.total_seconds())
    minutes, seconds = divmod(total_seconds, 60)
    return f"{minutes:02}:{seconds:02}"


# Apply formatting to both groups
avg_time_test_formatted = avg_time_test.apply(format_timedelta)
avg_time_control_formatted = avg_time_control.apply(format_timedelta)

In [ ]:
# Show the results
print("Average time spent on each step (Test group):")
print(avg_time_test)

print("\nAverage time spent on each step (Control group):")
print(avg_time_control)

Average time spent on each step (Test group):
process_step
confirm   0 days 00:04:10.566239316
start     0 days 00:01:01.454594512
step_1    0 days 00:01:00.756987249
step_2    0 days 00:01:28.878579881
step_3    0 days 00:02:09.607488325
Name: time_diff, dtype: timedelta64[ns]

Average time spent on each step (Control group):
process_step
confirm   0 days 00:03:00.146050670
start     0 days 00:01:06.804756575
step_1    0 days 00:00:50.535582680
step_2    0 days 00:01:32.043222569
step_3    0 days 00:02:17.224495833
Name: time_diff, dtype: timedelta64[ns]


In [34]:
# Display the results
print("Average time spent on each step (Test group):")
print(avg_time_test_formatted)

print("\nAverage time spent on each step (Control group):")
print(avg_time_control_formatted)

Average time spent on each step (Test group):
process_step
confirm    04:10
start      01:01
step_1     01:00
step_2     01:28
step_3     02:09
Name: time_diff, dtype: object

Average time spent on each step (Control group):
process_step
confirm    03:00
start      01:06
step_1     00:50
step_2     01:32
step_3     02:17
Name: time_diff, dtype: object
